In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_validate
from sklearn import linear_model, tree, ensemble
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from pprint import pprint



from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score



In [5]:
#trabajo sobre los conjuntos de datos ya preparados para la predicción
df_train = pd.read_excel("df_limpia.xlsx")



In [6]:
#analizo el número de registros por clase, porque después de probar con StratifiedKFold, tenía clases con un único registro, menor que los folds establecidos.


ytrain = df_train['service_code']
print(ytrain.value_counts())

4849671      9265
4849672      5835
4849669      4738
4849666      3396
5144577      2784
1000007      2366
5144587      1679
97550336     1278
5144579       846
9043968       814
5144576       798
1000008       741
1000000       526
1000010       474
9043969       436
1000012       420
5144578       311
5931009       301
7733248       291
1000002       247
103677952     230
97779712      217
4849670       215
5144585       206
1000017       181
5144582       147
4849673       134
118587392     134
188088321     122
1000029       119
1000009       114
1000014       112
1000016       107
82182147      102
4784129        99
5144586        75
5472256        72
1000011        67
5144581        65
1000030        65
40000          63
1000018        58
6029312        44
1000015        41
1000028        41
16777217       39
4849665        33
6520832        20
102793216      13
4849668        11
24182784        9
82182145        9
82182144        1
1000001         1
16777216        1
Name: serv

In [7]:
#me quedo para la predicción con las clases que tengan al menos 100 registros.
print(len(df_train))
df_train = df_train.groupby("service_code").filter(lambda x: len(x) >=99)
print(len(df_train))
#volvemos a definir ytrain
ytrain = df_train['service_code']

40513
39785


In [8]:
texting_corpus = df_train["texto_tratado"].str.split()


#genero una lista de listas para su vectorización del conjunto train
corpus_train=[[word for word in elem] for elem in texting_corpus ]
print(corpus_train[0:10])
print(len(corpus_train))

[['gente', 'venir', 'observar', 'excesivo', 'limpieza', 'centro', 'agua', 'llover', 'video', 'demostrar', 'necesario', 'agua', 'mayoria', 'gente', 'ciudad', 'ciudad', 'agua'], ['arreglar', 'echar', 'culpa', 'ciudadano', 'cargo', 'animal', 'sacar', 'necesidad', 'dejar', 'echar', 'verguenza', 'vecindario', 'palabra', 'limpieza', 'seguro', 'prestar', 'atencion', 'cercano', 'perro', 'llevar', 'semana', 'invitar', 'alcalde', 'pasear', 'disfrutar', 'acera', 'mobiliario', 'urbano', 'fachada', 'pis', 'continuo', 'perro', 'asqueroso', 'limpiar', 'fondo', 'multar', 'dueño', 'vecino', 'perro', 'tener', 'enviar', 'paseo'], ['bache', 'paseo', 'reparacion', 'bache'], ['bache', 'plaza', 'reparacion', 'bache'], ['bache', 'reparacion', 'bache'], ['bache', 'reparacion', 'bache'], ['unir', 'hecho', 'porqueria', 'entrada', 'peligroso', 'trabajador', 'empresa', 'hoja', 'mando', 'foto', 'hora'], ['limpiar', 'fondo', 'estar', 'plaza', 'alrededor'], ['malestar', 'vecino', 'lamentable', 'asfalto', 'grieta', 'p

In [9]:


'''
Obtenemos vectores para cada uno de los documentos con TfidfVectorizer que es una herramienta de vectorización de textos
que nó solo tiene en cuenta el nñumero de veces que aparece una palabra en un documento, sino su
peso o frecuencia a lo largo de todo el conjunto de documentos o corpus
'''
#construimos el modelo
modelo_tfidf_sklearn = TfidfVectorizer(max_df=0.5, min_df=5, use_idf= False)

#generamos los vectores para el conjunto train
xtrain =modelo_tfidf_sklearn.fit_transform([' '.join(doc) for doc in corpus_train])


In [12]:
#establezco 5 subconjuntos de datos formados con las mismas proporciones de clases que el conjunto inicial sobre los que estudiar los scores de los modelos

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=80)




cnt = 1

for train_index, test_index in kf.split(xtrain, ytrain):
    print(f'Fold:{cnt}, Subconjunto train: {len(train_index)}, Subconjunto test:{len(test_index)}')
    cnt+=1

Fold:1, Subconjunto train: 31828, Subconjunto test:7957
Fold:2, Subconjunto train: 31828, Subconjunto test:7957
Fold:3, Subconjunto train: 31828, Subconjunto test:7957
Fold:4, Subconjunto train: 31828, Subconjunto test:7957
Fold:5, Subconjunto train: 31828, Subconjunto test:7957


In [13]:
score = cross_val_score(linear_model.LogisticRegression(random_state= 80,max_iter=10000), xtrain, ytrain,   cv= kf, scoring="accuracy")
print(f'Scores para cada subconjunto: {score}')
print(f'Media del score: {"{:.2f}".format(score.mean())}')


#en este caso utilizo el accuracy que ya decíamos que no era el score adecuado para dataset desbalanceados, para comparar con el balanced_accuracy

Scores para cada subconjunto: [0.76674626 0.76586653 0.76360437 0.75970843 0.75744627]
Media del score: 0.76


In [15]:


#ahora como métrica utilizaremos la balanced_accuracy que tiene en cuenta el peso de las clases. Vemos que el accuracy cae respecto al cálculo anterior.

score = cross_val_score(linear_model.LogisticRegression(random_state= 80,max_iter=10000), xtrain, ytrain,  cv= kf, scoring="balanced_accuracy")
print(f'Scores para cada subconjunto: {score}')
print(f'Media del score: {"{:.2f}".format(score.mean())}')

Scores para cada subconjunto: [0.47057514 0.48742792 0.48816441 0.48442899 0.47233893]
Media del score: 0.48


In [23]:
#aqui lo que voy a hacer es valorar distintos modelos con metricas adecuadas también al desbalanceo
#incluyo la métrica balanced_accuracy_score

models = [RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state= 80,max_iter=10000),
]
names = ["Random_Forest", "Linear_SVC", "Multinomial_NB", "Logistic_Regression"]


# Elegimos las métricas que vamos a utilizar
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro')
}

for model, name in zip(models, names):
    # Realizamos también cross-validation con las métricas escogidas
    pprint(name)
    pprint(cross_validate(model, xtrain, ytrain, cv=5, scoring=scoring))

#Los mejores modelos, teniendo una baja accuracy ponderada son la Regresión Logística y Linear SVC

'Random_Forest'
{'fit_time': array([1.9296062 , 1.93007588, 2.20624042, 2.48741317, 2.0028832 ]),
 'score_time': array([0.32898045, 0.3468051 , 0.44497252, 0.3452189 , 0.34538531]),
 'test_accuracy': array([0.33643333, 0.32474551, 0.31846173, 0.32864145, 0.33190901]),
 'test_balanced_accuracy': array([0.06169634, 0.05853957, 0.05731845, 0.06023441, 0.06055291]),
 'test_f1_score': array([0.05158015, 0.04780056, 0.04642406, 0.04869245, 0.05133773]),
 'test_recall': array([0.33643333, 0.32474551, 0.31846173, 0.32864145, 0.33190901])}
'Linear_SVC'
{'fit_time': array([3.64542532, 2.32608151, 3.28913116, 2.97012734, 1.99333715]),
 'score_time': array([0.02503061, 0.02291322, 0.02554417, 0.02516341, 0.02409792]),
 'test_accuracy': array([0.73532738, 0.75719492, 0.76586653, 0.75480709, 0.74186251]),
 'test_balanced_accuracy': array([0.4868317 , 0.52449768, 0.56021312, 0.51783007, 0.49055693]),
 'test_f1_score': array([0.49761963, 0.54351092, 0.58341236, 0.53734287, 0.50460418]),
 'test_recall'